<figure>
  <img src="../images/tudelft_logo.png" alt="image" width="250" align="right"/>
</figure>

# 1b: Process-based coastal classification

Welcome to the second notebook of week 1. In the previous notebook, you have looked at tectonic classification. We now continue with process-based coastal classification. In this notebook, we will explore several coastal systems around the world considering the second- and third-order scale of features (i.e., at regional $O$(10-100km) and local $O$(0.1-1km) scales).

In Chapter 2 of the [Coastal Dynamics Open Textbook](https://books.open.tudelft.nl/home/catalog/book/202), it is explained how coastal systems can be classified according to the processes that influence these systems. For example, one of the figures (below) shows how the relative influence of fluvial, wave and tidal processes determines the shape of coastal features. The goal of this notebook exercise is that you can identify the signatures of the processes that are introduced in Chapter 2 in several coastal systems around the world. Looking at remotely-sensed (satellite or aerial) images of coastal systems and interpreting which physical processes shape the coast is an important skill for the modern coastal engineer.

For this exercise, we will first prepare the visualization of the coastal systems. Next, you will be asked to answer 13 questions, mainly multiple-choice or multiple-selection questions, and 1 numeric question.

<!-- ![image](./database/1_coastal_classification//1_coastal_forcing.png) -->

<img src="../images/1_coastal_classification//1_coastal_forcing.png" alt="image" width="550"/>

## Import libraries that we use for our analysis

Let's first import the libraries that we use for our analysis by running the next cells.

In [ ]:
from pathlib import Path
import warnings

import geopandas as gpd
import holoviews as hv
import hvplot.pandas  # noqa: API import
import numpy as np
import pandas as pd
import panel as pn
import pooch
from bokeh.models import PanTool, WheelZoomTool

import coastal_dynamics as cd

# Activate Panel extension to make interactive visualizations
pn.extension()

In [ ]:
# read questions locally
questions = cd.read_questions(Path("../hashed_questions/1_coastal_classification_hashed.json"))

question_industry = cd.QuestionIndustry(questions)

cd.UseAnswersApp("1b").serve()

## Load the coastal systems data and prepare the visualization

### Load coastal systems data
In this notebook, we will take a closer look at individual coastal systems. To make that a bit easier, we have prepared a dataset of coastal sites with their coordinates. You might not recognize the '.parquet' file extension but don't worry, it's functionally similar to '.csv' (like Excel), but allows us to save and load geospatial data using geopandas. Load this dataset by running the cell below.

In [ ]:
coastal_systems_fp = Path("../database/1_coastal_classification/1_coastal_systems.parquet")

coastal_systems = gpd.read_parquet(coastal_systems_fp)

### Load the isobaths

Like in notebook 1a, we also get the bathymetric contours for a water depth of -200m from [ArcGIS](https://www.arcgis.com/home/item.html?id=5f98dbc4708e4a7e99c0a7fe043d70a1), which we will use as a proxy to find the boundary of the continental shelf. Because we want to maintain interactive plots, all shorter isobaths are dropped. Computing lengths is a metric operation, so the data first has to be reprojected from a geographic coordinate system in degrees latitude/longitude to a planar projection.  

In [ ]:
isobath_fp = Path("../database/1_coastal_classification/1_isobaths200.gpkg")

data200 = gpd.read_file(isobath_fp)
data200["length"] = data200.to_crs("EPSG:3857").geometry.length 
data200 = data200[data200["length"] > 5 * 10**6]

 ### Define the plot function for the coastal systems data

In the cell below we define a small plot function that generates an ESRI World Imagery base map given a longitude, latitude, zoom level and name. A small dataframe of coastal systems around the world is also loaded into `geopandas`, a Python library for geospatial tabular data. In the cells afterwards, we sample this dataframe and show the coastal system on a map. 

**Note**: Although you don't have to understand the plot method, we include it here so you can see how these interactive plots are made! 

In [ ]:
def show_coastal_systems():
    """
    Creates an app showing coastal systems.
    """

    # Below we build the widget
    title_bar = pn.Row(
        pn.pane.Markdown(
            "## Coastal systems overview",
            styles={"color": "black"},
            width=800,
            sizing_mode="fixed",
            margin=(10, 5, 10, 15),
        ),
        pn.Spacer(),
    )
    
    options = coastal_systems.name.to_list()
    coastal_systems_slider = pn.widgets.Select(
        name="Coastal system", options=options, value=np.random.choice(options)
    )

    plot_isobaths = pn.widgets.Select(
        name="Plot isobaths -200m?", options=["no", "yes"]
    )

    @pn.depends(coastal_systems_slider.param.value, plot_isobaths.param.value)
    def plot_coastal_system(name, plot_isobath):
        system = coastal_systems.loc[coastal_systems["name"] == name].copy()
        west, south, east, north = system[
            ["west", "south", "east", "north"]
        ].values.flatten()

        p = system.hvplot.points(
            x="lon",
            y="lat",
            geo=True,
            color="red",
            alpha=0,
            xlim=(west, east),
            ylim=(south, north),
            tiles="ESRI",
            frame_width=1100,
            ylabel="Latitude [deg]",
            xlabel="Longitude [deg]",
        )

        if plot_isobath == "yes":
            baths = data200.hvplot(
                geo=True, line_width=2, line_color="white", line_dash="dashed"
            )
            p = p * baths

        p.opts(frame_width=1000, frame_height=500, tools=["pan", "wheel_zoom"])

        return p

    app = pn.Column(
        title_bar,
        pn.Row(plot_isobaths, align="center"),
        pn.Row(coastal_systems_slider, align="center"),
        pn.Row(plot_coastal_system, align="center"),
    )

    return app

### Plot the coastal systems data

Execute the cell below to plot the coastal systems data using the function we defined above. Please note that altering the slider positions or selecting different options from the dropdown menus may trigger a warning; it can safely be ignored, and possibly silenced by adjusting the logging warning level.

In [ ]:
app = show_coastal_systems()

cd.launch_app(app)

## Questions

By running the below cells you get two blocks of in total 13 questions. You can use the Coastal Systems panel in combination with Chapter 2 of your textbook to answer the questions. 
The coastal systems from the drop-down menu in the panel correspond to the ones classified in Table 2.3 of the textbook. Compare these classifications with evidence from the satellite images for each system. The below questions are designed to help you think along in this way.

Please note that time always plays a role in classification. A previously sediment-rich system for instance, may be sediment-poor in current times. Systems may also show relict features from the past when there was a different balance between physical processes. For example, you can find beach/dune relicts in barrier islands on the rim of the Mississippi delta system (Figure 2.46 in the textbook). These islands date back to a previous era with larger sediment supply.

### Question block 1: characterizing Individual Coastal Systems
One of the most useful skills that a modern coastal engineer can have is being able to interpret satellite imagery. What can we learn about coasts from remote sensing? How can we apply what we learned in Chapter 2 about coastal classification? When you look at these images, consider the following checklist:
 - Where is your site with respect to plate boundaries? (leading, trailing)
 - Is there evidence of past RSLR rise, fall, or stability?
 - Is there evidence of past glaciation?
 - What are the dominant hydrodynamic processes and sediment sources (wave/tide/river)?
 - Primary/secondary coastal forms? (See Table 2.3)
 - What kind of sediment is present (sand/mud/other)? How much?
 - What kind of biotic or human influences do you see?
 - Is the coast prograding or retreating?

Let's first consider some individual coastal systems. Have a look at the system mentioned in the questions below, then try to answer the questions based on the satellite images and what you have learned so far. HINT: Zoom in and out to see the wider context of each site, as well as the smaller details of the coast.

In [ ]:
q = [
    "Q1b-columbia_and_ganges",
    "Q1b-mekong",
    "Q1b-wadden_sea",
    "Q1b-wax_lake",
    "Q1b-wadden_sea_vegetation",
    "Q1b-st_michel",
    "Q1b-willapa_bay",
]

question_industry.serve(q)

### Question block 2: Application
You have hopefully learned a lot this week about coastal classification at different spatial scales. Let's now take a step back. The questions below cover some more application-oriented topics and not all questions concern coastal classification. Some questions ask you to compare different systems, and others might ask you about stuff you have not seen before in this course. You will need to engage your critical thinking skills to answer them correctly, and we will take a deeper dive into these topics in the coming weeks. Have fun!

In [ ]:
q = [
    "Q1b-dune_du_pilat",
    "Q1b-engineered_delta",
    "Q1b-scale_lagoa_de_albofeira_and_amazon",
    "Q1b-sandy_and_muddy_sediment",
    "Q1b-pearl_and_st_michel",
    "Q1b-favourite_site",
]

question_industry.serve(q)

### The end 

You have reached the end of this Notebook 1b. This was the last notebook of this week.